In [4]:
import cv2
import time
import numpy as np
import hand_detector as hd
import pyautogui

In [5]:
wCam, hCam = 640, 480
frameR = 100
smoothening = 7

pTime = 0
plocX, plocY = 0, 0
clocX, clocY = 0, 0

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
detector = hd.handDetector(detectionCon=0.7)
wScr, hScr = pyautogui.size()
print(wScr, hScr)

2880 1800


In [ ]:
while True:
    success, img = cap.read()  # 카메라에서 프레임을 읽음
    img = detector.findHands(img)  # 손을 감지하여 표시
    lmList, bbox = detector.findPosition(img)  # 손의 위치 정보를 가져옴
    output = img.copy()  # 이미지 복사본 생성

    if len(lmList) != 0:
        # print(lmList[4], lmList[8])
        x1, y1 = lmList[8][1:]  # 검지 끝 좌표
        x2, y2 = lmList[12][1:]  # 중지 끝 좌표

        fingers = detector.fingersUp()  # 각 손가락이 올라갔는지 확인
        # print(fingers)
        cv2.rectangle(img, (frameR, frameR), (wCam - frameR, hCam - frameR), (205, 250, 255), -1)  # 화면 경계 사각형
        img = cv2.addWeighted(img, 0.5, output, 1 - .5, 0, output)  # 화면에 투명도 추가

        # 검지만 들어올려진 경우: 이동 모드
        if fingers[1] == 1 and fingers[2] == 0:
            # 좌표 변환
            x3 = np.interp(x1, (frameR, wCam - frameR), (0, wScr))
            y3 = np.interp(y1, (frameR, hCam - frameR), (0, hScr))

            # 이동 값을 부드럽게 처리
            clocX = plocX + (x3 - plocX) / smoothening
            clocY = plocY + (y3 - plocY) / smoothening

            # 마우스 이동
            pyautogui.moveTo(wScr - clocX, clocY)
            cv2.circle(img, (x1, y1), 6, (255, 28, 0), cv2.FILLED)  # 이동 모드 표시
            plocX, plocY = clocX, clocY
            # cv2.putText(img, 'Moving Mode', (20, 50), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)

        # 검지와 중지가 둘 다 들어올려진 경우: 클릭 모드
        if fingers[1] == 1 and fingers[2] == 1:
            # 두 손가락 사이 거리 측정
            length, img, lineInfo = detector.findDistance(8, 12, img)

            # 거리 짧으면 클릭
            if length < 40:
                cv2.circle(img, (lineInfo[4], lineInfo[5]), 6, (0, 255, 0), cv2.FILLED)  # 클릭 표시
                # cv2.putText(img, 'Click!!', (20, 50), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
                pyautogui.click()

    cTime = time.time()  # 현재 시간
    fps = 1 / (cTime - pTime)  # 프레임 속도 계산
    pTime = cTime

    cv2.imshow("Vitual mouse monitor", cv2.flip(img, 1))  # 화면을 좌우 반전하여 표시
    cv2.setWindowProperty("Vitual mouse monitor", cv2.WND_PROP_TOPMOST, 1)  # 창을 항상 위로 설정
    cv2.waitKey(1)

    if cv2.waitKey(1) & 0xFF == 27:  # Esc 키가 눌리면 종료
        break

cap.release()  # 카메라 해제
cv2.destroyAllWindows()  # 모든 창 닫기
